In [ ]:
import pandas as pd
import pandas_gbq as pgbq
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
folder = '/content/drive/My Drive/Colaboração/AI ecosystem - LatAm/'

Mounted at /content/drive/


In [ ]:
areas_sources = pd.read_excel(folder+'Comunidades de IA.xlsx')

In [ ]:
pgbq.to_gbq(areas_sources,project_id='insyspo',destination_table='projectdb_ciaam.areas_communities')

100%|██████████| 1/1 [00:00<00:00, 5262.61it/s]


In [ ]:
%%bigquery --project=insyspo

CREATE OR REPLACE TABLE projectdb_ciaam.areas_works AS (
  SELECT DISTINCT a.area, b.source_id, b.work_id
  FROM projectdb_ciaam.areas_communities AS a
  JOIN publicdb_openalex_2025_08_rm.works_locations AS b
  ON a.id = b.source_id
)

Query is running:   0%|          |

""


In [ ]:
%%bigquery --project=insyspo

CREATE OR REPLACE TABLE projectdb_ciaam.sources_works_authors AS (
  SELECT a.area, a.source_id, b.work_id, b.author_id
  FROM projectdb_ciaam.areas_works AS a
  JOIN publicdb_openalex_2025_08_rm.works_authorships AS b
  ON a.work_id = b.work_id
);

CREATE OR REPLACE TABLE projectdb_ciaam.sources_authors AS (
  SELECT DISTINCT area, source_id, author_id
  FROM projectdb_ciaam.sources_works_authors
);


CREATE OR REPLACE TABLE projectdb_ciaam.sources_n_authors AS (
  SELECT area, source_id,
    COUNT(DISTINCT author_id) AS authors,
    COUNT(DISTINCT work_id) AS works
  FROM projectdb_ciaam.sources_works_authors AS a
  GROUP BY area, source_id
);

CREATE OR REPLACE TABLE projectdb_ciaam.level0_distinct_authors AS (
  SELECT DISTINCT author_id
  FROM projectdb_ciaam.sources_authors
);

Query is running:   0%|          |

""


In [ ]:
%%bigquery --project=insyspo

CREATE OR REPLACE TABLE projectdb_ciaam.areas_sources AS (
  SELECT DISTINCT area, other_source_id AS source_id
  FROM (
    SELECT
      area,
      source_id,
      other_source_id,
      authors,
      ROW_NUMBER() OVER (PARTITION BY source_id ORDER BY authors DESC) AS r
    FROM (
      SELECT a.area,
        a.source_id,
        c.source_id AS other_source_id,
        COUNT(DISTINCT a.author_id) AS authors
      FROM projectdb_ciaam.sources_authors AS a
      JOIN publicdb_openalex_2025_08_rm.works_authorships AS b
      ON a.author_id = b.author_id
      JOIN publicdb_openalex_2025_08_rm.works_locations AS c
      ON b.work_id = c.work_id
      #WHERE a.author_id > b.author_id
      #  AND a.source_id > b.source_id
      GROUP BY a.area, a.source_id, c.source_id
    )
  )
  WHERE r <=4
);

Query is running:   0%|          |

""


In [ ]:
%%bigquery --project=insyspo

CREATE OR REPLACE TABLE projectdb_ciaam.areas_works_all AS (
  SELECT DISTINCT a.area, b.source_id, b.work_id
  FROM projectdb_ciaam.areas_sources AS a
  JOIN publicdb_openalex_2025_08_rm.works_locations AS b
  ON a.source_id = b.source_id
);

CREATE OR REPLACE TABLE projectdb_ciaam.all_sources_works_authors AS (
  SELECT a.area, a.source_id, b.work_id, b.author_id
  FROM projectdb_ciaam.areas_works AS a
  JOIN publicdb_openalex_2025_08_rm.works_authorships AS b
  ON a.work_id = b.work_id
);

Query is running:   0%|          |

""


In [ ]:
%%bigquery df --project=insyspo

SELECT b.display_name, a.*, c.IDLattes
FROM (
  SELECT author_id, area, COUNT(DISTINCT work_id) AS works
  FROM projectdb_ciaam.all_sources_works_authors
  GROUP BY author_id, area
) AS a
JOIN publicdb_openalex_2025_08_rm.authors AS b
ON a.author_id = b.id
JOIN projectdb_lattes.match_researchers_lattes_openalex AS c
ON a.author_id = c.author_id
JOIN projectdb_lattes_2025.total_Tabelaconsolidada AS e
ON c.IDLattes = e.IDLattes
WHERE EndProfPais = 'Brasil' AND Graduacao >= '1' #AND works_topic>=3
ORDER BY works DESC

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df

,display_name,author_id,area,works,IDLattes
0,Benjamín Bedregal,5004002970,Fuzzy,74,'4601263005352005
1,Teresa B. Ludermir,5025550530,Neural,40,'8510946198843716
2,Teresa B. Ludermir,5025550530,Neural,40,'6321179168854922
3,Liang Zhao,5100697945,Neural,39,'1531713258988427
4,Graçaliz Pereira Dimuro,5053196020,Fuzzy,37,'9414212573217453
...,...,...,...,...,...
1782,Elder Donizetti dos Santos,5036569395,Fuzzy,1,'9468214941700162
1783,Ricardo M. Leão,5038966561,Neural,1,'8125613125649746
1784,Rosiane Rodrigues,5011819011,Neural,1,'8358219976594707
1785,Moacir Antonelli Ponti,5029845097,Visão,1,'5041497500746910


In [ ]:
%%bigquery --project=insyspo
SELECT DISTINCT author_id
FROM (
SELECT b.display_name, a.*, c.IDLattes
FROM (
  SELECT author_id, area, COUNT(DISTINCT work_id) AS works
  FROM projectdb_ciaam.all_sources_works_authors
  GROUP BY author_id, area
) AS a
JOIN publicdb_openalex_2025_08_rm.authors AS b
ON a.author_id = b.id
JOIN projectdb_lattes.match_researchers_lattes_openalex AS c
ON a.author_id = c.author_id
ORDER BY works DESC)

Query is running:   0%|          |

Downloading:   0%|          |

,author_id
0,5111805527
1,5092105215
2,5080429951
3,5085001789
4,5016801444
...,...
2436,5103040672
2437,5032033063
2438,5064564799
2439,5032685263


In [ ]:
df.to_csv('autores_ia_br_comunidades.csv')

In [ ]:
areas = list(df['area'].unique())

In [ ]:
pd.pivot(df,values=df['author_id'],columns=areas)

KeyError: 'Multi-agentes'

In [ ]:
%%bigquery --project=insyspo

CREATE OR REPLACE TABLE projectdb_ciaam.authors_connections_0 AS (
  SELECT a.author_id, b.author_id,
    c.source_id, COUNT(DISTINCT b.work_id) AS works
  FROM projectdb_ciaam.level0_distinct_authors AS a
  JOIN publicdb_openalex_2025_08_rm.works_authorships AS b
  ON a.author_id = b.author_id
  JOIN publicdb_openalex_2025_08_rm.works_locations AS c
  ON b.work_id = c.work_id
  WHERE a.author_id > b.author_id
  GROUP BY a.author_id, c.source_id
);

Query is running:   0%|          |

""


In [ ]:
%%bigquery --project=insyspo

CREATE OR REPLACE TABLE projectdb_ciaam.authors_connections_0 AS (
  SELECT a.author_id, c.source_id, COUNT(DISTINCT b.work_id) AS works
  FROM projectdb_ciaam.authors_connections_0 AS a
  JOIN publicdb_openalex_2025_08_rm.works_locations AS c
  ON b.work_id = c.work_id
  JOIN publicdb_openalex_2025_08_rm.works_authorships AS b
  ON a.author_id = b.author_id
  JOIN publicdb_openalex_2025_08_rm.works_locations AS c
  ON b.work_id = c.work_id
  GROUP BY a.author_id, c.source_id
);

In [ ]:
%%bigquery --project=insyspo

#CREATE OR REPLACE TABLE projectdb_ciaam.areas_works AS (
  SELECT DISTINCT a.area, b.source_id, c.work_id
  FROM projectdb_ciaam.areas_works AS a
  JOIN publicdb_openalex_2025_08_rm.works_authorships AS b
  ON a.work_id = b.work_id
  JOIN publicdb_openalex_2025_08_rm.works_authorships AS c
  ON b.author_id = c.author_id
  JOIN publicdb_openalex_2025_08_rm.works_locations AS d
  ON c.work_id = d.work_id

#)